In [ ]:
import pandas as pd
import random
import requests
import json
import os

# --- Hằng số và siêu tham số Q-learning ---
LEARNING_RATE = 0.1
DISCOUNT_FACTOR = 0.9
EXPLORATION_RATE = 0.2
MOODLE_URL = 'http://localhost:8100/webservice/rest/server.php'
TOKEN = '84cffdbb9ead18d97ccc45f9889bc926'  # token của local user log (custome api)

ACTIONS = [
    'read_new_resource',
    'review_old_resource',
    'attempt_new_quiz',
    'redo_failed_quiz',
    'skip_to_next_module',
    'do_quiz_harder',
    'do_quiz_easier',
    'do_quiz_same'
]

# --- Phân loại score và complete_rate thành bin (lưu mốc dưới) ---
SCORE_AVG_BINS = [0, 2, 4, 6, 8]     # 5 bin: 0-2,2-4,4-6,6-8,8-10
COMPLETE_RATE_BINS = [0.0, 0.3, 0.6] # 3 bin: 0-0.3,0.3-0.6,0.6-1.0

 # Load JSON
with open('json_course_moodle_hierarchy_clean.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Lấy tất cả sectionId_new từ lessons
section_ids = []
for section in data:
    if "lessons" in section:
        for lesson in section["lessons"]:
            section_ids.append(lesson["sectionId_new"])
            

# --- Q-table ---
q_table = {}

# --- Hàm khởi tạo Q-table ---
def initialize_q_table(filename='q_table_results.csv'):
    initial_q_table = {}
    levels = range(1, 4)     # level 1-3

    for section_id in section_ids:
        for level in levels:
            for complete_rate in COMPLETE_RATE_BINS:
                for score_bin in SCORE_AVG_BINS:
                    state = (section_id, level, complete_rate, score_bin)
                    initial_q_table[state] = {action: 0.0 for action in ACTIONS}

    save_q_table_to_csv(initial_q_table, filename)
    return initial_q_table

# --- Lưu Q-table ra CSV ---
def save_q_table_to_csv(q_table_to_save, filename='q_table_results.csv'):
    q_table_data = []
    for state, actions_dict in q_table_to_save.items():
        for action, q_value in actions_dict.items():
            row = {
                'sectionid': state[0],
                'level': state[1],
                'complete_rate': state[2],
                'score_bin': state[3],
                'action': action,
                'q_value': q_value
            }
            q_table_data.append(row)
    df = pd.DataFrame(q_table_data)
    df.to_csv(filename, index=False)

# --- Load Q-table từ CSV ---
def load_q_table_from_csv(filename='q_table_results.csv'):
    q_table_loaded = {}
    if not os.path.exists(filename):
        return {}

    df = pd.read_csv(filename)
    for _, row in df.iterrows():
        state = (row['sectionid'], row['level'], row['complete_rate'], row['score_bin'])
        action = row['action']
        q_value = row['q_value']

        if state not in q_table_loaded:
            q_table_loaded[state] = {a: 0.0 for a in ACTIONS}

        q_table_loaded[state][action] = q_value
    return q_table_loaded

# --- Rời rạc hóa score 0-10 ---
def discretize_score(score):
    for b in reversed(SCORE_AVG_BINS):
        if score >= b:
            return b
    return SCORE_AVG_BINS[0]

# --- Rời rạc hóa complete rate 0-1 ---
def discretize_complete_rate(rate):
    for b in reversed(COMPLETE_RATE_BINS):
        if rate >= b:
            return b
    return COMPLETE_RATE_BINS[0]


# --- Hàm lấy Cluster của người dùng từ file CSV ---
def get_user_cluster(user_id):
    """
    Lấy thông tin cluster của người dùng từ file CSV.
    """
    file_path = "./synthetic_user_features_clustered.csv"
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print("❌ Không tìm thấy file: synthetic_user_features_clustered.csv")
        return None
    user_row = df[df['userid'] == user_id]
    if user_row.empty:
        return None
    return user_row.iloc[0]['cluster']


# --- Hàm gọi Moodle API ---
def call_api(function, extra_params):
    """
    Gọi Moodle Web Service API.
    """
    params = {
        'wstoken': TOKEN,
        'moodlewsrestformat': 'json',
        'wsfunction': function
    }
    params.update(extra_params)
    try:
        response = requests.post(MOODLE_URL, data=params)
        response.raise_for_status() # Kiểm tra lỗi HTTP
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"❌ Lỗi khi gọi Moodle API '{function}': {e}")
        return {} # Trả về dictionary rỗng để tránh lỗi

# --- Lấy Q-value ---
def get_q_value(state, action):
    global q_table
    if state not in q_table:
        q_table[state] = {a: 0.0 for a in ACTIONS}
    return q_table[state][action]

# --- Cập nhật Q-table ---
def update_q_table(state, action, reward, next_state):
    global q_table
    current_q = get_q_value(state, action)
    max_future_q = max(q_table[next_state].values()) if next_state in q_table else 0.0
    q_table[state][action] = current_q + LEARNING_RATE * (reward + DISCOUNT_FACTOR * max_future_q - current_q)

def get_reward(action, old_score, new_score, old_complete, new_complete,
               cluster=None, quiz_level='medium', passed_hard_quiz=True):
    """
    Tính reward cho từng hành động của học sinh trong một section cụ thể.
    Thêm logic cho quiz level: easy, medium, hard.
    """

    # reward cơ bản theo cải thiện điểm
    score_improvement = new_score - old_score

    # reward theo cải thiện complete rate
    complete_bonus = (new_complete - old_complete) * 10  # scale để rõ ràng

    # reward theo action
    action_bonus = 0
    if action == 'read_new_resource':
        action_bonus = 1
    elif action == 'review_old_resource':
        action_bonus = 0.5
    elif action == 'attempt_new_quiz':
        # thưởng nhiều hơn nếu quiz khó
        if quiz_level == 'easy':
            action_bonus = 1.5
        elif quiz_level == 'medium':
            action_bonus = 2
        elif quiz_level == 'hard':
            action_bonus = 3
    elif action == 'redo_failed_quiz':
        action_bonus = 2.5
    elif action == 'skip_to_next_module':
        action_bonus = -1
    elif action == 'do_quiz_harder':
        action_bonus = 3
    elif action == 'do_quiz_easier':
        action_bonus = 0.5
        if not passed_hard_quiz:
            action_bonus += 5  # khuyến khích chuyển sang dễ nếu không pass
    elif action == 'do_quiz_same':
        action_bonus = 1

    # reward cá nhân hóa theo cluster
    cluster_bonus = 0
    if cluster == 0: # yếu
        if action in ['review_old_resource', 'redo_failed_quiz']:
            cluster_bonus = 2
        if action == 'do_quiz_easier':
            cluster_bonus += 1  # khuyến khích học sinh yếu chuyển sang dễ
    elif cluster == 1: # mạnh, giỏi
        if action == 'do_quiz_harder':
            cluster_bonus = 2

    # reward cá nhân hóa tổng
    personal_bonus = 0

    reward = score_improvement + complete_bonus + action_bonus + cluster_bonus + personal_bonus
    return reward

# --- Chọn action ---
def suggest_next_action(current_state):
    if current_state not in q_table or random.uniform(0, 1) < EXPLORATION_RATE:
        action = random.choice(ACTIONS)
        q_value = get_q_value(current_state, action)
        return action, q_value
    else:   
        q_values = q_table[current_state]
        best_action = max(q_values, key=q_values.get)
        return best_action, q_values[best_action]

def getStateFromMoodle(userid, courseid, sectionid):
    """
    Lấy state hiện tại của user trong 1 section cụ thể từ Moodle API.
    
    Trả về:
    - state: tuple (avg_quiz_score, completion_rate, score_bin, complete_bin)
    - quiz_passed: True/False (pass quiz khó hay không)
    """

    # --- 1. Điểm trung bình quiz của user trong section/course ---
    avg_quiz_score_data = call_api('local_userlog_get_avg_quiz_score', {
        'userid': userid,
        'courseid': courseid
    })
    avg_quiz_score = avg_quiz_score_data.get('avg_quiz_score', 0.0)

    # --- 2. Tài nguyên trong section ---
    total_resource_data = call_api('local_userlog_get_total_resources_by_section', {
        'sectionid': sectionid,
        'objecttypes[0]': 'resource',
        'objecttypes[1]': 'hvp',  # giữ lại nếu HVPs dùng được
    })
    total_resource = total_resource_data.get('total_resources', 0)

    # --- 3. Tài nguyên đã xem ---
    viewed_resource_data = call_api('local_userlog_get_viewed_resources_distinct_by_section', {
        'userid': userid,
        'courseid': courseid,
        'sectionid': sectionid,
        'objecttypes[0]': 'resource',
        'objecttypes[1]': 'hvp',
    })
    viewed_resource = viewed_resource_data.get('viewed_resources', 0)

    # --- 4. Tính tỷ lệ hoàn thành section ---
    completion_rate = viewed_resource / total_resource if total_resource > 0 else 0

    # --- 5. Kiểm tra user pass quiz khó hay không ---
    quiz_passed_data = call_api('local_userlog_get_latest_quiz_pass_status_by_section', {
        'sectionid': sectionid,
        'userid': userid,
    })
    # Nếu API trả về 'is_passed' = 1 → True, 0 → False
    passed_hard_quiz = quiz_passed_data.get('is_passed', 0) == 1

    # --- 6. Rời rạc hóa điểm và complete rate ---
    score_bin = discretize_score(avg_quiz_score)
    complete_bin = discretize_complete_rate(completion_rate)

    # --- 7. Trả về state ---
    state = (sectionid, 1, complete_bin, score_bin)  # Level tạm set =1, có thể lấy động nếu có API
    return state, completion_rate, avg_quiz_score, passed_hard_quiz


# --- Demo sử dụng ---
if __name__ == '__main__':
    q_table = load_q_table_from_csv()
    if not q_table:
        q_table = initialize_q_table()

    idUser = 4  # Giả sử idUser là 4
    cluster_user = get_user_cluster(idUser)  # lấy cluster của user từ file CSV
    quiz_level = 'medium'  # giả lập quiz đang làm
    passed_hard_quiz = False  # giả lập chưa pass bài khó

    # Ví dụ state cũ
    old_state = (43, 2, discretize_complete_rate(0.25), discretize_score(7))
    action, qv = suggest_next_action(old_state)
    print(f"💡 Suggest action: {action} (Q={qv:.2f})")

    # Giả lập next state
    new_state = (43, 2, discretize_complete_rate(0.5), discretize_score(8))

    # Tính reward
    reward = get_reward(
        action=action,
        old_score=7,
        new_score=8,
        old_complete=0.25,
        new_complete=0.5,
        cluster=cluster_user,
        quiz_level=quiz_level,
        passed_hard_quiz=passed_hard_quiz
    )
    print(f"🏆 Reward: {reward}")

    # Update Q-table
    update_q_table(old_state, action, reward, new_state)
    save_q_table_to_csv(q_table)



💡 Suggest action: read_new_resource (Q=4.10)
🏆 Reward: 4.5
